In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

def main():
    spark = (
        SparkSession.builder
        .appName("IcelandWindStreamingSkeleton")
        .getOrCreate()
    )

    # TODO: update Kafka bootstrap servers and topic name
    df = (
        spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "vm1:9092")
        .option("subscribe", "weather_iceland_raw")
        .option("startingOffsets", "latest")
        .load()
    )

    # Just print raw JSON for now
    lines = df.selectExpr("CAST(value AS STRING) as json_str")

    query = (
        lines.writeStream
        .outputMode("append")
        .format("console")
        .option("truncate", False)
        .start()
    )

    query.awaitTermination()

if __name__ == "__main__":
    main()
